In [ ]:
import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os.path

from scipy.stats import norm, t
from IPython.core.display import display

%matplotlib inline

sns.set()

plt.rcParams['font.family'] = 'Yu Mincho'

# C:\sample\neural_net_simulation に保存
WORK_DIR = os.path.join('C:\\', 'sample', 'neural_net_simulation')

ソースコード4.39　ダウ・ジョーンズデータの取込

In [ ]:
dji = pd.read_csv(
    os.path.join(WORK_DIR, 'data', 'DJI.csv'), 
    parse_dates=['Date'],
    engine='python'
)

ソースコード4.40　ログリターンの計算

In [ ]:
dji = dji.assign(ログリターン=np.log(dji.loc[:, 'Adj Close']).diff())
dji = dji.iloc[1:]

ソースコード4.41　ログリターンのヒストグラムとフィットした正規分布のプロット

In [ ]:
sns.distplot(dji.loc[1:, 'ログリターン'], fit=norm)

ソースコード4.42　ログリターンのヒストグラムとフィットしたt分布のプロット

In [ ]:
sns.distplot(dji.loc[1:, 'ログリターン'], fit=t)

* ソースコード4.43: ニューラルネットワークの入力と期待出力（バッチ考慮後）（IndexSimulationNet クラスから抜粋）
* ソースコード4.44　 バッチ標準化付きニューラルネットワークの隠れ層（IndexSimulationNetクラスから抜粋）
* ソースコード4.45　 隠れ層からドリフト項とボラティリティ項を作成し、分布オブジェクトでまとめる（IndexSimulationNetクラスから抜粋）
* ソースコード4.46　平均尤度の定義（IndexSimulationNetクラスから抜粋）
* ソースコード4.47　学習メソッド（IndexSimulationNetクラスから抜粋）

In [ ]:
class IndexSimulationNet:
    def __init__(
        self,
        sess,
        batch_size,
        lookback_size,
        num_hidden_neurons,
        num_hidden_layers,
        learning_rate, 
        momentum=0.8,
        training=False
    ):
# --- ソースコード4.43 ---
        with tf.variable_scope('input_layer'):
            self._inputs = tf.placeholder(tf.float32, [batch_size, lookback_size], name='inputs')
            self._labels = tf.placeholder(tf.float32, [batch_size, 1], name='label')
# --- ソースコード4.43 ここまで ---            

        with tf.variable_scope('hidden_layers'):
# --- ソースコード4.44 ---
            self._inputs_normed = tf.layers.batch_normalization(
                tf.expand_dims(self._inputs, axis=-1),
                momentum=momentum,
                training=training
            )
            self._hidden_layer = tf.squeeze(self._inputs_normed, axis=-1)
            
            for layer in range(num_hidden_layers):
                self._hidden_layer = tf.layers.dense(
                    self._hidden_layer, 
                    num_hidden_neurons,
                    activation=tf.tanh
                )
# --- ソースコード4.44 ここまで ---
                
        with tf.variable_scope('distribution'):
# --- ソースコード4.45 ---
            self._scale = tf.nn.softplus(
                tf.layers.dense(
                    self._hidden_layer,
                    1,
                    name='scale'
                )
            )  # ボラティリティ項。ソフトプラス関数を最後にかける。
            
            self._loc = tf.layers.dense(
                self._hidden_layer, 1, name='loc')
            self._distr = tf.distributions.StudentT(df=3., loc=self._loc, scale=self._scale)
# --- ソースコード4.45 ここまで ---
            
        with tf.variable_scope('outputs'):
# --- ソースコード4.46  ---
            self._sample_size = tf.get_variable(
                'sample_size', 1, dtype=tf.int32, trainable=False
            )
            # 分布からサンプリング
            self._sample = self._distr.sample(self._sample_size)
        
            # 対数尤度の平均
            self._log_likelihood = tf.reduce_mean(
                tf.log(
                    self._distr.prob(self._labels) 
                )
            )
            update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
            
            # バッチ標準化の更新ステップを追加
            with tf.control_dependencies(update_ops):
                self._training_step = tf.train.AdamOptimizer(
                    learning_rate=learning_rate
                ).minimize(
                    -self._log_likelihood  # 対数尤度にマイナスをかけたものを最小化
                )
# --- ソースコード4.46 ここまで ---
        
        if training:
            sess.run(tf.global_variables_initializer())
        
        self._saver = tf.train.Saver()
        
    def train_one_step(self, sess, inputs, labels):
# --- ソースコード4.47 ---
        fetches = [
            self._training_step,
            self._log_likelihood
        ]
        feed_dict = {
            self._inputs: inputs,
            self._labels: labels
        }
        return sess.run(fetches, feed_dict)
# --- ソースコード4.47 ここまで ---
    
    def sample(self, sess, inputs, sample_size=(1,)):
        return sess.run(
            fetches=[
                self._sample,
                self._scale,
                self._loc,
            ],
            feed_dict={
                self._inputs: inputs,
                self._sample_size: sample_size
            }
        )
    
    def save(self, sess, path):
        self._saver.save(sess, path)
        print(f'model saved in {path}')
        
    def restore(self, sess, path):
        self._saver.restore(sess, path)
        print(f'model restored from {path}')


ソースコード4.48　ニューラルネットワークのメタパラメータを固定

In [ ]:
lookback_size = 32
batch_size = 32
num_hidden_neurons = 128
num_hidden_layers = 3
max_learn_steps = 20000
polling_interval = 500
learning_rate = 0.003

ソースコード4.49　シミュレーション用ニューラルネットワークの学習

In [ ]:
likelihoods_over_time = np.empty(max_learn_steps // polling_interval)
tf.reset_default_graph()

with tf.Graph().as_default() and tf.Session() as session:
    index_sim_net = IndexSimulationNet(
        session, batch_size, lookback_size,
        num_hidden_neurons, num_hidden_layers, learning_rate,
        training=True
    )
    
    likelihood_mean_polling_interval = 0.
    
    for learn_step in range(max_learn_steps):
        random_row_indices = np.random.choice(
            dji.shape[0] - lookback_size - 1, size=batch_size, replace=True
        )
        random_rows = [
            dji.iloc[random_row:random_row + lookback_size + 1, :] 
            for random_row in random_row_indices
        ]
        log_returns_for_this_step = np.reshape(
           [row[['ログリターン']].values for row in random_rows],
            (batch_size, lookback_size + 1)
        )
        
        _, log_likelihood = index_sim_net.train_one_step(
            session,
            log_returns_for_this_step[:, :lookback_size],
            log_returns_for_this_step[:, lookback_size:]
        )
        likelihood_mean_polling_interval += log_likelihood
        if learn_step % polling_interval == 0:
            likelihoods_over_time[
                learn_step // polling_interval
            ] = likelihood_mean_polling_interval / polling_interval
            
            likelihood_mean_polling_interval = 0.
            print(f'[{(learn_step * 100) // max_learn_steps:3}%] '
                  f'直近{polling_interval}ステップの平均尤度：'
                  f'{likelihoods_over_time[learn_step // polling_interval]:.5f}')
            
    index_sim_net.save(session, os.path.join(WORK_DIR, 'models', 'index_sim_dji'))
likelihoods_over_time_copy = likelihoods_over_time.copy()

In [ ]:
# 学習ステップに対しての平均尤度
fig, ax = plt.subplots(figsize=(12, 5))
sns.lineplot(x=list(range(likelihoods_over_time.shape[0])), y=likelihoods_over_time, ax=ax)

ソースコード4.50　ログリターン用の箱の用意

In [ ]:
num_sim_steps = 100
sim_start_index = np.random.randint(
    0,
    dji.shape[0] - lookback_size - num_sim_steps
)
real_log_returns = dji.iloc[
    sim_start_index:sim_start_index + lookback_size + num_sim_steps, :
][['ログリターン']].values.flatten()
gen_log_returns = real_log_returns.copy()
path_dates = dji.iloc[
    sim_start_index:sim_start_index + lookback_size + num_sim_steps, :
][['Date']].values.flatten()

ソースコード4.51　一つのパスのシミュレーション

In [ ]:
gen_scales = np.empty(num_sim_steps)
gen_locs = np.empty_like(gen_scales)
gen_dfs = np.empty_like(gen_scales)

sim_batch_size = 1

print(f'{pd.to_datetime(path_dates[lookback_size]):%Y-%m-%d}'
      f' ～ {pd.to_datetime(path_dates[~0]):%Y-%m-%d}'
      ' のログリターンをシミュレーションします。')
tf.reset_default_graph()
with tf.Graph().as_default() and tf.Session() as sess:
    index_sim_net = IndexSimulationNet(
        session,
        sim_batch_size,
        lookback_size,
        num_hidden_neurons,
        num_hidden_layers,
        learning_rate,
        training=False
    )
    index_sim_net.restore(
        sess,
        os.path.join(WORK_DIR, 'models', 'index_sim_dji')
    )
    
    rolling_log_returns = np.array(
        [real_log_returns[:lookback_size]]
    )
    for sim_step in range(num_sim_steps):
        sample, scale, loc = index_sim_net.sample(
            sess,
            rolling_log_returns,
            [1])
        rolling_log_returns[:, :~0] = rolling_log_returns[:, 1:]
        rolling_log_returns[:, ~0] = sample
        
        gen_log_returns[lookback_size + sim_step] = sample
        gen_scales[sim_step] = scale
        gen_locs[sim_step] = loc
    print(f'{num_sim_steps}個のログリターンをシミュレーションしました。')

ソースコード4.52　指数値の計算

In [ ]:
both_log_returns = pd.DataFrame(
    data=np.transpose([gen_log_returns, real_log_returns]),
    columns=['シミュレーション値', '実際値'],
    index=path_dates
)
asset_values = np.exp(both_log_returns).cumprod().stack().reset_index()
asset_values.columns = ['日付', 'タイプ', 'アセット価値']

ソースコード4.53　指数値のパスのプロット

In [ ]:
fig, ax = plt.subplots(figsize=(12, 5))
sns.lineplot(data=asset_values, x='日付', y='アセット価値', hue='タイプ', ax=ax)